In [2]:
import torch_geometric.datasets as datasets
from torch_geometric.loader import DataLoader
from lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
import torch
import wandb

from gnnexplain.nn.model import GC2GNNModel

cora = datasets.Planetoid(
    root='data',
    name='Cora',
    split='random',
    num_train_per_class=200,
    num_test=200,
    num_val=200
)

loader = DataLoader(cora, batch_size=2, shuffle=True, num_workers=4)

torch.set_float32_matmul_precision('medium')

model = GC2GNNModel(cora.num_features, 32, cora.num_classes, 4, 4)
logger = WandbLogger(project="gnnexplain", group="cora")

trainer = Trainer(
    max_epochs=100,
    log_every_n_steps=1,
    logger=logger
)
trainer.fit(model, train_dataloaders=loader)
wandb.finish()

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/apluska/miniconda3/envs/gnnexplain/lib/python3.12/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lexpk. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name         | Type   | Params
----------------------------------------
0 | model        | GC2GNN | 71.5 K
  | other params | n/a    | 1     
----------------------------------------
71.5 K    Trainable params
2         Non-trainable params
71.5 K    Total params
0.286     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  6.33it/s, v_num=qee1, train_loss=1.380, regularizer=0.337]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  5.47it/s, v_num=qee1, train_loss=1.380, regularizer=0.337]


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lamb * regularizer,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
regularizer,▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train_loss,██████████▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,99
lamb * regularizer,0.0
regularizer,0.33696
train_loss,1.38466
trainer/global_step,99


In [15]:
[p for p in model.parameters()]

[Parameter containing:
 tensor(0.),
 Parameter containing:
 tensor(1.1000),
 Parameter containing:
 tensor([[-0.0471,  0.0685,  0.1811,  ...,  0.0121,  0.1176,  0.0887],
         [-0.1022,  0.1363,  0.1763,  ...,  0.1372,  0.1524, -0.0609],
         [ 0.1889, -0.0301,  0.1712,  ..., -0.0850, -0.0309,  0.2203],
         ...,
         [ 0.1059,  0.2340, -0.0269,  ..., -0.0471,  0.0361,  0.1084],
         [-0.0349, -0.1163,  0.0632,  ...,  0.0135, -0.1455, -0.1484],
         [ 0.1175, -0.0073,  0.1375,  ...,  0.1024,  0.1778, -0.0799]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0568,  0.1265,  0.1342, -0.0450, -0.1017,  0.1617,  0.1220, -0.0198,
          0.0825,  0.2008,  0.0788,  0.1509,  0.0609, -0.0667,  0.1383,  0.0879,
          0.1314, -0.1153,  0.1931,  0.0714, -0.1471, -0.1743,  0.0054, -0.1356,
          0.0676, -0.0689,  0.0783,  0.0540,  0.1771,  0.1807, -0.1027,  0.0505],
        requires_grad=True),
 Parameter containing:
 tensor([[ 0.0774,  0.0312,  0.

In [19]:
model.model.blocks[3].hidden_layers[3].regularizer()

tensor(434.8303, grad_fn=<SumBackward0>)